Imports

In [ ]:
!pip install ffmpeg-python
!pip install av
!pip install --upgrade pip
!pip install --upgrade transformers datasets[audio] accelerate
!pip install openai

In [ ]:
import os
import cv2
import time
import json
import torch
import random
import ffmpeg
import warnings
import numpy as np
import pandas as pd
from PIL import Image
from glob import glob
import soundfile as sf
from openai import OpenAI
from json import loads,dumps
import matplotlib.pyplot as plt
from scipy.signal import resample
import typing_extensions as typing
from google.generativeai.types import HarmCategory, HarmBlockThreshold

In [ ]:
# Extracting all required ids 
labels_df = pd.read_csv('/kaggle/input/dynamic-few-shot/filtered_ground_truth.csv') 

transcriptions_df = pd.read_csv('/kaggle/input/youtube-data/eng-complete-transcriptions.csv')
df = pd.merge(labels_df, transcriptions_df, on='Video Id', how='inner') 

df 

In [ ]:
# Filtering out available ads and those that are either inappropriate or child directed

df = df[(df['Primary Label'] == 'inappropriate') | (df['Primary Label'] == 'appropriate')] 
df['Primary Label'].value_counts().plot(kind='bar') 

In [ ]:
# Extracting video ids and primary labels

video_ids = list(df['Video Id'])
primary_labels = list(df['Primary Label'])
all_transcriptions = list(df['Transcription'])

In [ ]:
# Extracting data from transcripts

transcriptions = []
lengths = []

for (i, id_) in enumerate(video_ids):
    transcriptions.append(all_transcriptions[i].split("chunks")[0])
    lengths.append(len(all_transcriptions[i].split("chunks")[0]))


In [ ]:
import os 

available_ids = os.listdir('/kaggle/input/youtube-data/Ads/Ads') 
len(available_ids) 

In [ ]:
apikey = ""

client = OpenAI(api_key=apikey) 

In [ ]:
output_schema = {
  "type": "object",
  "properties": {
    "label": {
      "type": "string",
      "enum": ["inappropriate", "appropriate"]
    },
    "justification": {
      "type": "string",
      "minLength": 10
    }, 
    "languages": {
        "type": "array", 
        "items": {
            "type": "string" 
        }
        
    }
  },
  "required": ["label", "justification", "languages"]
} 

In [ ]:
import torch
print(torch.cuda.is_available())

Prompt 

In [ ]:
prompt =  """"A video can be considered inappropriate for children if it contains physical violence (cartoonish, realistic, or gory), interpersonal violence (bullying, pranks, meanness, belittling, controlling behavior, talking down to others, or manipulation), self-harm or suicide (depictions of harm inflicted on oneself or suicidal thoughts/tendencies), extreme stunts (life-endangering, high-risk activities/challenges that require adult supervision), dangerous products or services (like paintball, airsoft, fireworks, weapons, hunting equipment, graffiti products), scary content (horror, suspense, loud violence, zombies, skeletons, masks, scary clowns, blood, dangerous fire, car crashes, medical procedures or other scary visuals), sexual content (innuendos, sexual behavior, nudity, suggestiveness), intimacy (romantic conduct, couples expressing love, sensual contact, dating, relationships or any expression of love or loss for a significant other,  lyrics about or mentions of love or romance, heartbreak), inappropriate clothing (significant skin exposure or sheer clothing outside of appropriate contexts such as sunbathing, swinmwear,etc), substance use (mention, depictions, or use of alcohol, drugs, tobacco), gambling (betting, gambling, casinos, online or real world casino games, online trading), offensive language (from mild to severe with words such as “stupid” or “idiot”), stereotypes (racial, ethnic, gender, body shaming), crude humor (inappropriate humor, innuendos), weapons (guns, swords, knives, etc but toy guns such as nerf guns are fine), unsuitable media (ads for violent or adult-themed video games, casino games, games with romance or battle or first person-shooter games), fight sports (boxing, wrestling, martial arts), financial content (difference, rolling spot forex, financial spread betting and synonymous products but banking ads are fine), health and beauty services (cosmetic procedures, weight loss, tanning, piercings, tattoos, cosmetics and makeup (no matter their essential benefits) and other products related to external personal care focused on body image but ads on fashion products and essential hygiene products like sunscreen, shampoo, toothpaste, etc. are fine, however, products like sanitary care and hair removal is inappropriate), ads or sponsored content for consumable food or drink products (regardless of nutritional content), where the product is consumed by a person or shown with branding, packaging, or promotional intent are inappropriate; general content like cooking shows, recipes, or scenes where food is simply present or being eaten without promotion are acceptable, pet/animal food products and healthcare, healthcare or medical topics (reproductive health, mental health, substance abuse, eating disorders, pharmaceuticals, health insurance, any nutritional or dietary supplements), sensitive topics (politics, religion, controversial social issues, astrology, occult, paranormal, religious sermons), privacy or safety risks (services or competitions requiring personal information, social networks for strangers, spy services, or video game skin trading), or any explicit depictions or discussions of death or dying. Consider that you are an expert content moderator. Based on the audio transcription, is the video advertisement provided inappropriate for children on this definition? If it is, label it as “inappropriate.” If it is not, label it as “appropriate.” Justify your conclusion. Also, mention what language is being spoken if there is any. 
Make the outputs in JSON format: {“label”: “appropriate” or “inappropriate”, “justification”: “brief explanation of the classification”, “language”: “language detected in the video”}."""

prompt 

In [ ]:
start = 0
end = len(video_ids)

In [ ]:
import requests

def classify_video_with_text(text_input, audio_transcription):
    """
    Sends text and audio transcription to the API for classification.

    Parameters:
    - text_input: str, the input text or prompt.
    - audio_transcription: str, transcription of the audio.

    Returns:
    - response: The API response.
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {apikey}"
    }

    payload = {
        "model": "gpt-4o-2024-08-06",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": text_input},
                    {"type": "text", "text": 'Audio transcription: ' + audio_transcription},
                ]
            }
        ],
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "output_schema",
                "schema": output_schema
            }
        },
        "max_tokens": 300,
        "temperature": 0.0
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    return response

In [ ]:
ids = []
labels = []
responses = []
predicted_labels = [] 
languages = [] 
remaining = [] 


for i in range(start,end): 

    if video_ids[i] in available_ids:

        try:
            # Upload audio transcription 
            audio = transcriptions[i] 

            # Make inference
            try:
                classification_response = classify_video_with_text(prompt, audio)
            
                temp_id = video_ids[i] 
                temp_label = primary_labels[i] 
                temp_predicted_label = json.loads(classification_response.json()['choices'][0]['message']['content']).get('label') 
                temp_response = classification_response.json()['choices'][0]['message']['content'] 
                temp_languages = json.loads(classification_response.json()['choices'][0]['message']['content']).get('languages') 

                ids.append(temp_id)
                labels.append(temp_label)
                predicted_labels.append(temp_predicted_label)
                responses.append(temp_response)
                languages.append(temp_languages) 

                print(i)

                print('\nPrimary Label:', primary_labels[i], '. Response:', classification_response.json()['choices'][0]['message']['content'])
                
            except Exception as e:
                print(f"Error processing response.text: {e}")
                remaining.append(video_ids[i])
                continue

        except Exception as e:
            print(f"Unexpected error: {e}")
            remaining.append(video_ids[i])
            continue

        # Wait a bit to avoid exceeding rate limits
        time.sleep(25)

# At the end, print remaining videos
print("Remaining videos with errors:", remaining)

In [ ]:
remaining_df = pd.DataFrame({'Video Id': remaining}) 
remaining_df.to_csv(f'remaining-native-gpt-{start}-{end}.csv', index=False) 

In [ ]:
for i in range(len(responses)): 
    print('True Label: ', primary_labels[i], '\tPrediction: ', predicted_labels[i]) 

In [ ]:
new_df = pd.DataFrame({
    'Video Id': ids,
    'Primary Label': labels,
    'Predicted Label': predicted_labels,
    'Response': responses, 
    'Languages': languages 
})

new_df.head() 

In [ ]:
results_dir = '/kaggle/working/results'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

In [ ]:
new_df.to_csv(f'/kaggle/working/results/gpt-native_audios_only_{start}-{end}.csv', index=False) 

In [ ]:
# Changing to binary lists 

predictions = [1 if pred == 'inappropriate' else 0 for pred in predicted_labels] 
ground_truths = [1 if label == 'inappropriate' else 0 for label in labels] 

In [ ]:
# Obtaining classification report 
from sklearn.metrics import classification_report 

report = classification_report(ground_truths, predictions) 
print(report) 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(ground_truths, predictions)

plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Appropriate', 'Inapproriate'], yticklabels=['Appropriate', 'Inapproriate'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()